In [ ]:
%matplotlib notebook

# This examples shows who to download files from the ONC server
import glob
import os

import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import strawb
import strawb.sensors.pmtspec
import strawb.tools

import plotly
import plotly.graph_objs as go
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_white"

import random

plt.rcParams.update({"text.usetex": False})  # fix time fmt of x_ticks

# Get a working file

Get file from DB and df5 attribute measurement_type

In [ ]:
# in case execute db.load_entire_db_from_ONC() to load the entire db
db = strawb.SyncDBHandler(file_name='Default')  # loads the db

In [ ]:
# mask by device
mask = db.dataframe['deviceCode'] == 'TUMPMTSPECTROMETER001'

mask &= db.dataframe['dataProductCode'] == 'PMTSD'

# and add mask by 'measurement_type'
mask &= db.dataframe['synced']

# and add mask by 'measurement_type'
# mask &= db.dataframe['measurement_type'] == 'laser_adjustment'

db.dataframe[mask]  # plot the table

## In case none of the files is synced, download some files

In [ ]:
mask = db.dataframe['deviceCode'] == 'TUMPMTSPECTROMETER001'
mask &= db.dataframe['dataProductCode'] == 'PMTSD'

## select 1h frame around one biolumi event
timestamp = np.datetime64('2021-09-04T23:44:09')
mask &= db.dataframe.dateFrom >= pd.Timestamp(np.datetime64(timestamp, "h"), tz="UTC")
mask &= db.dataframe.dateFrom < pd.Timestamp(np.datetime64(timestamp, "h"), tz="UTC") + np.timedelta64(1, "h")

### show selected file from the DB - here its 1 file only
db.dataframe[mask]

In [ ]:
### this will download the selected file
if not db.dataframe.synced[mask].all():
    db.update_db_and_load_files(
        db.dataframe[mask],
        output=True,  # print output to console
        download=True,  # download the files
        #save_db=True
    )
    db.save_db()

this is now all included in db.update_db_and_load_files()


    if mask.sum() < 5:
        onc_downloader = strawb.ONCDownloader()
        onc_downloader.download_structured(db.dataframe[~mask_download].iloc[-5:])
        print('\n Download done. Update DB.')
        db.update_sync_state()
        db.update_hdf5_attributes()
        db.save_db()
    else:
        print(f'{mask_download.sum()} files exists, no download.')

## Helper function

In [ ]:
def get_working_file_randomly(file_list, file_version=1):
    """Some files fails with the file_handler, i.e., if a group is missing for older SDAQ versions.
    To get one file randomly which works, try it until it works.
    """

    file_list_trials = file_list.copy()
    while len(file_list_trials) > 0:
        i = np.random.randint(0,len(file_list_trials))
        full_path = file_list_trials[i]
        try:
            spec = strawb.sensors.pmtspec.FileHandler(full_path)
        except Exception as a:
            file_list_trials.pop(i)
            print(f'Error: {full_path} failed with: {a}')
        else:
            if spec.file_version >= file_version:
                print(f'Success: {full_path}, file_version={spec.file_version}')
                return full_path
            else:
                file_list_trials.pop(i)
                print(f'Error: {full_path}, file_version: {spec.file_version}')
            
    return None

## Get a working file

In [ ]:
file_version = 1  # select one of [1, 2, 3]

file_list = db.dataframe[mask]['fullPath'].to_list()

full_path = get_working_file_randomly(file_list, file_version)  # select a random file

print(f'Use: {full_path}')

## Load the file

In [ ]:
pmtspec = strawb.sensors.PMTSpec(file=full_path)
pmtspec.file_handler.file_attributes  # show the file attributes, time in seconds since epoch (1.1.1970) UTC

# Set up plotting function

In [ ]:
def multiline_plot(data_frame):
    columns = list(set(data_frame.columns).difference(['time']))
    
    fig = plotly.subplots.make_subplots(shared_xaxes=True,
                                        rows=len(columns),
                                        cols=1, 
                                        #subplot_titles=columns
                                       )


    fig.update_layout(
        width=800,
        height=150+100*len(columns),)

    j = 1
    for i, column_i in enumerate(columns):
        fig.add_trace(go.Scatter(x=data_frame['time'], 
                                 y=data_frame[column_i],
                                 name=column_i), 
                      row=i+1, col=1
                     )

    return fig

# Plotting with pandas and plotly

In [ ]:
df_daq = pmtspec.file_handler.get_pandas_daq()
df_hv = pmtspec.file_handler.get_pandas_hv()
df_padiwa = pmtspec.file_handler.get_pandas_padiwa()

df_counts = pmtspec.file_handler.get_pandas_counts()

df_dcounts = pmtspec.trb_rates.get_pandas_dcounts()
df_rates = pmtspec.trb_rates.get_pandas_rate()

# Basic plots direct from file

In [ ]:
fig = multiline_plot(df_daq)
fig.show()

In [ ]:
fig = multiline_plot(df_hv)
fig.show()

In [ ]:
fig = multiline_plot(df_padiwa)
fig.show()

# Counts and Rates

In [ ]:
# Don't plot counter data, it's less but 100Hz has some size.
# fig = multiline_plot(df_counts)
# fig.show()

In [ ]:
# # Don't plot TOT data, it's a lot of data
# fig = multiline_plot(df_tot)
# fig.show()

# dcounts are the diff of the stored TRB counts readings. As the readings can jitter a bit, dcounts_pmt jitter, too
# to correct device by dcounts_time to get a rate (or take df_rates) or 
# by dcounts_laser to get a success rate [events/trigger]
df_counts.shape

In [ ]:
# Moreover those values have to be processed further. -> df_rates
# fig = multiline_plot(df_rates)
# fig.show()

df_rates.shape

# Select a couple of seconds for plotting PMT rates

In [ ]:
### plot the PMT spectra using their correct wavelength filter color
from strawb.tools import wavelength_to_rgb

In [ ]:
%matplotlib inline

In [ ]:
### this is the event we found in dev_camera_basic_notebook.ipynb
timestamp = np.datetime64('2021-09-04T23:44:09')

### use a nice colormap, or use wavelength_to_rgb for conversion
pmtspec.pmt_meta_data.add_colors(
        plt.cm.get_cmap('viridis'),
        None,
        plt.cm.get_cmap("gist_heat") # reddish color
    )

times_ns = strawb.tools.asdatetime(pmtspec.trb_rates.time, 'ns')
### select a couple of seconds around biolumi events
## you can adjust this time for zoom-in/out
timemask = times_ns >= timestamp + np.timedelta64(15, "s") # - np.timedelta64(2, "m")
timemask &= times_ns <= timestamp + np.timedelta64(30, "s")
masked_times_ns = times_ns[timemask]

plt.figure(figsize=(15,6))
plt.title(timestamp)
for ch in pmtspec.pmt_meta_data.channel_meta_array:
    _ = plt.plot(
        masked_times_ns,
        pmtspec.trb_rates.rate.T[timemask][:,ch["index"]], 
        color = wavelength_to_rgb(ch, gamma=0.4),
        label = ch["label"],
        marker="o", lw=0, alpha=0.3, ms=5
    )
plt.legend(loc=2, ncol=2)
# that's when the camera frame starts
#plt.axvline(timestamp, lw=1, color="k")
plt.ylim(0, np.max(pmtspec.trb_rates.rate.T[timemask])*1.05)